## Módulos 

In [49]:
#Módulo de Caixa preta --> Algoritmo Simplex

from mip import *



# Módulo para criar o modelo e resolver o problema de programação linear

def save(model, filename):
    
    # Função para salvar o modelo em um arquivo .lp e imprimir na tela
    
    model.write(filename) 
    with open(filename, "r") as f: 
        print(f.read())



def solve(model):
    
    # Função para resolver o modelo e imprimir na tela
    
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value:.2f}\n")

    print("Solution:")
    for v in model.vars:
        print(f"{v.name} = {v.x:.2f}")


def CreateModel(QtdVar,QtdRes,z,restricoes):
    
    # Função para criar o modelo usando dados retirados da instância
    # Recebe strings da função 
    # Retorna o modelo criado
    print("Criando modelo...")
    print(f"Qtd de Variaveis: {QtdVar}, Qtd de Restricoes: {QtdRes}")    
    
    
    model = Model(sense=MAXIMIZE, solver_name=CBC)    
    x = {i: model.add_var(var_type=BINARY, name=f'x_{i}', lb=0.0) for i in range(QtdVar)}    
    
    model.objective = xsum(z[i]*x[i] for i in range(QtdVar)) # Max(z1*x1 + z2*x2 + ... + zn*xn))    
    
    # S.A:
    for j in range(QtdRes):        
       model += xsum(restricoes[j][i]*x[i] for i in range(QtdVar)) <= restricoes[j][QtdVar]    
       
    
    
    save(model, "modelo.lp")
    solve(model)
    return model
    
    
    
    

def info(model):
    
    print("Informações do modelo:")
    
    print("Z = ", model.objective_value)
    
    print("Valores das variáveis:")
    print("x = ", [x.x for x in model.vars])
        


In [3]:
#Modulos para leitura de arquivos

def read_file(file_name):
    
    #Funçao destinada a ler arquivo das instancias
    #recebe como parametro o nome do arquivo
    #retorna a quantidade de variaveis, quantidade de restriçoes, vetor de custos e matriz de restriçoes
    
    file = open(file_name, 'r')
    lines = file.readlines()
    file.close()
    
    # Separa as informações e converte para int
    QtdVar,QtdRes = [int(i) for i in lines[0].strip().split(' ')]       
    z = [int(i) for i in lines[1].strip().split(' ')]
    restricoes = [[int(j) for j in i.strip().split(' ')] for i in lines[2:]]
    
    
    return QtdVar,QtdRes,z,restricoes





In [4]:
def Bound(model,limitante):
    
    # Função tem como objetivo restringir as ramificações do modelo, podando conforme as trÊs restrições:
    # 1 - Poda por integralidade
    # 2 - Poda por inviabilidade
    # 3 - Poda por limitante
    
    # Recebe o modelo Atual
    # Retornar Booleano para indicar se pode ou não podar o modelo daquele ramo 
    
    
    #Verificar se todas as variaveis são inteiras
    if all([i.x.is_integer() for i in model.vars]): #  Maneira mais elegante de verificar se a divisão inteira por 1 ira resultar em um numero inteiro para todas as variaveis
        print("Poda por integralidade")
        return True

    #Verificar se o modelo é inviavel -> Não tem solução
    if model.num_solutions == 0:
        print("Poda por inviabilidade")
        return True
    
    #Verificar se o valor da função objetivo é menor que o limitante
    if model.objective_bound <= limitante:
        print("Poda por limitante")
        return True
    
    
    

In [45]:
def Branch(model):
    
    #       Função tem como objetivo ramificar o modelo (escolhendo uma variável cujo valor fracionário sejá mais próximo de 0.5 )
    ##      Criando dois novos modelos  um para xi = 0 e outro para xi = 1
    # Recebe o modelo Atual
    # Retornar os dois novos modelos criados
    
    minimum = 0.51 # Variavel para guardar o valor fracionario mais proximo de 0.5
    resto = 0
    varIndex = 0 # Variavel para guardar o indice da variavel que possui o valor fracionario mais proximo de 0.5
    
    
    
    for i in range(len(model.vars)):
        resto = model.vars[i].x % 1
        if resto > 0.5:
            resto = 1 - resto
        if resto < minimum:
            minimum = resto
            varIndex = i
            
            
    #model.branch_up = lambda self, i: self.branch(var_fix={i: 1})
    #model.branch_down = lambda self, i: self.branch(var_fix={i: 0})
    
    print(f"Variavel escolhida para ramificar: {varIndex}")
    print(f"Valor fracionario: {model.vars[varIndex].x}")
    print(f"Valor fracionario mais proximo de 0.5: {minimum}")
    
    #copiar modelo atual e criar dois novos modelos, com as restrições de xi = 0 e xi = 1 adicionadas
    
    #Sub-arvore da esquerda
    modelEsquerda = model.copy()
    modelEsquerda += modelEsquerda.vars[varIndex] == 0
    
    
    
    #Sub-arvore da direita
    modelDireita = model.copy()
    modelDireita += modelDireita.vars[varIndex] == 1
    
    modelEsquerda.optimize()
    modelDireita.optimize()
    
    return modelEsquerda,modelDireita
    
    
    
    
    
    

## Testando Instâncias

In [6]:
r = read_file("I0.txt")
m = CreateModel(r[0],r[1],r[2],r[3])



Criando modelo...
Qtd de Variaveis: 3, Qtd de Restricoes: 2
\Problem name: 

Minimize
OBJROW: -5 x_0 - x_1
Subject To
constr(0):  3 x_0 + 5 x_1 + 2 x_2 <= 6
constr(1):  4 x_0 + 4 x_1 + 4 x_2 <= 7
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
Integers
x_0 x_1 x_2 
End

Status =  OptimizationStatus.OPTIMAL
Solution value  = 5.00

Solution:
x_0 = 1.00
x_1 = 0.00
x_2 = 0.00
